In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

import logging
import sys
from IPython.display import Markdown

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [63]:
from llama_index import ListIndex, LLMPredictor
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.schema import MetadataMode
from llama_index.response import pprint_utils as pp

In [4]:
llm_predictor = LLMPredictor(
    llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=512)
)

In [5]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
    MetadataFeatureExtractor,
)
from llama_index.langchain_helpers.text_splitter import SentenceSplitter


text_splitter = SentenceSplitter(
  chunk_size=512,
  chunk_overlap=0,
)

In [6]:
metadata_extractor = MetadataExtractor(
    extractors=[
        SummaryExtractor(summaries=["self"]),
        KeywordExtractor(keywords=10),
    ],
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

In [7]:
from llama_index import SimpleDirectoryReader, DocumentSummaryIndex

In [8]:
from pathlib import Path

documents_ho3 = SimpleDirectoryReader(input_files=['./data/HO3_sample.pdf']).load_data()

In [9]:
nodes_ho3 = node_parser.get_nodes_from_documents(documents_ho3)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyri...
> Adding chunk: HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyri...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: involving such vehicle or craft.
b.For the purp...
> Adding chunk: involving such vehicle or craft.
b.For the purp...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: is received,other than the mutual exchange of s...
> Adding chunk: is received,other than the mutual exchange of s...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Page 2 of 22 Copyright, Insurance Services Offi...
> Adding chunk: Page 2 of 22 Copyright, Insurance Services Offi...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: an "insured"; and
(2)Where an "insured" is temp...
> Adding chunk: an "insured"; and
(2)Where an "insured" is temp...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: household or domestic services;or
b.One who per...
> Adding chunk: h

In [10]:
nodes_ho3[0].metadata

{'page_label': '1',
 'file_name': 'HO3_sample.pdf',
 'section_summary': '\nThis section of the policy outlines the agreement between the company providing the insurance and the named insured. It also defines certain words and phrases used in the policy, such as Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, and Watercraft Liability. These definitions include the liability for bodily injury or property damage arising from the ownership, maintenance, occupancy, operation, use, loading, or unloading of the vehicle or craft, as well as the entrustment of the vehicle or craft to any person, the failure to supervise or negligent supervision of any person involving the vehicle or craft, and the vicarious liability for the actions of a child or minor involving the vehicle or craft.',
 'excerpt_keywords': ' Homeowners, Insurance, Agreement, Definitions, Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, Watercraft Liability, Model Aircraft, Hobby Aircraft'}

In [11]:
nodes_ho3[1].metadata

{'page_label': '1',
 'file_name': 'HO3_sample.pdf',
 'section_summary': '\nThis section defines the terms "aircraft", "hovercraft", "watercraft", "motor vehicle", "bodily injury", "business", "employee", and "insured". It outlines the activities that constitute a business, and the types of employees that are covered by the policy. It also states that residents of the insured\'s household are also covered.',
 'excerpt_keywords': '\nInsurance, Bodily Injury, Business, Aircraft, Hovercraft, Watercraft, Motor Vehicle, Compensation, Employee, Residence Employee.'}

In [12]:
print(nodes_ho3[0].text)

HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM
AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A.In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.
B.In addition, certain words and phrases are definedas follows:
1."Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a.Liability for "bodily injury" or "property dam-age" arising out of the:
(1)Ownership of such vehicle or craft by an"insured";
(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;
(3)Entrustment of such vehicle or craft b

In [13]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import FaissVectorStore
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

In [14]:
print(
    "LLM sees:\n",
    (nodes_ho3)[0].get_content(metadata_mode=MetadataMode.LLM),
)

LLM sees:
 [Excerpt from document]
page_label: 1
file_name: HO3_sample.pdf
section_summary: 
This section of the policy outlines the agreement between the company providing the insurance and the named insured. It also defines certain words and phrases used in the policy, such as Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, and Watercraft Liability. These definitions include the liability for bodily injury or property damage arising from the ownership, maintenance, occupancy, operation, use, loading, or unloading of the vehicle or craft, as well as the entrustment of the vehicle or craft to any person, the failure to supervise or negligent supervision of any person involving the vehicle or craft, and the vicarious liability for the actions of a child or minor involving the vehicle or craft.
excerpt_keywords:  Homeowners, Insurance, Agreement, Definitions, Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, Watercraft Liability, Model Aircraft, Hobby 

In [15]:
index = VectorStoreIndex(nodes=nodes_ho3)
engine = index.as_query_engine(similarity_top_k=10)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["[Excerpt from document] page_label: 1 file_name: HO3_sample.pdf section_summary:  This section of the policy outlines the agreement between the company providing the insurance and the named insured. It also defines certain words and phrases used in the policy, such as Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, and Watercraft Liability. These definitions include the liability for bodily injury or property damage arising from the ownership, maintenance, occupancy, operation, use, loading, or unloading of the vehicle or craft, as well as the entrustment of the vehicle or craft to any person, the failure to supervise or negligent supervision of any person involving the vehicle or craft, and the vicarious liability for the actions of a ch

In [16]:
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, node_parser=node_parser
)
question_gen = LLMQuestionGenerator.from_defaults(
    service_context=service_context,
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

In [17]:
DEFAULT_SUB_QUESTION_PROMPT_TMPL

'Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\n\n# Example 1\n<Tools>\n```json\n{{\n    "uber_10k": "Provides information about Uber financials for year 2021",\n    "lyft_10k": "Provides information about Lyft financials for year 2021"\n}}\n```\n\n<User Question>\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\n\n\n<Output>\n```json\n[\n    {{\n        "sub_question": "What is the revenue growth of Uber",\n        "tool_name": "uber_10k"\n    }},\n    {{\n        "sub_question": "What is the EBITDA of Uber",\n        "tool_name": "uber_10k"\n    }},\n    {{\n        "sub_question": "What is the revenue growth of Lyft",\n        "tool_name": "lyft_10k"\n    }},\n    {{\n        "sub_question": "What is the EBITDA of Lyft",\n        "tool_name": "lyft_10k"\n    }}\n]\n```\n\n# Example 2\n<Tools>\n```json\n{tools_str}\n```\n\n<User Question>\n{query_str}\n\n

In [18]:
final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="homeowners_policy_documents",
                description="insurance policy contract detailing coverage provisions.",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=True,
)

In [19]:
response = final_engine.query(
    """
    How does the policy addresses water damage for dwelling damage and personal property damage?
    Include a bullet point list of citations in your answer.
    """
)
print(response.response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["\\n        Follow the example, but instead of giving a question, always prefix the question \\n        with: \'By first identifying and quoting the most relevant sources, \'. \\n        Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\

In [71]:
pp.pprint_response(response, show_source=True)

Final Response: Yes, the policy does cover water damage. However,
there are some conditions and restrictions. The coverage applies to
direct physical loss to property caused by various types of water
damage, including flood, surface water, waves, tidal water, overflow
of a body of water, or spray from any of these. It also covers water
or water-borne material that backs up through sewers or drains,
overflows or is discharged from a sump or related equipment, or water
or water-borne material below the surface of the ground.   However,
the coverage does not apply to the loss of the system or appliance
from which the water or steam escaped, or to certain fixtures or
equipment such as sump pumps, roof drains, gutters, downspouts, or
similar fixtures. Additionally, if the building is protected by an
automatic fire protective sprinkler system, the water supply and heat
must be maintained for coverage to apply. Losses caused by mold,
fungus, or wet rot must result from an accidental discharge

In [20]:
print(response.response)

The homeowners policy documents address water damage for dwelling damage and personal property damage as follows: 

• For dwelling damage, water damage is covered under Section I – Property Coverages A. Coverage A – Dwelling, including losses from water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment; or water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure. Direct loss by fire, explosion or theft resulting from water damage is also covered. Exclusions from this coverage include losses caused by flood, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind, as well as losses caused by ordinance or law, war or military action, nuclear hazard, government action, earth movement,

In [21]:
response = final_engine.query(
    """
    How does the policy addresses water damage for dwelling damage and personal property damage?
    """
)
print(response.response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["\\n        Follow the example, but instead of giving a question, always prefix the question \\n        with: \'By first identifying and quoting the most relevant sources, \'. \\n        Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\

In [22]:
print(response.response)

The homeowners policy document states that water damage for dwelling damage is covered under Section I – Property Coverages A. Coverage A – Dwelling. It states that the dwelling on the "residence premises" shown in the Declarations, including structures attached to the dwelling, is covered. It also states that water damage means flood, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind; water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment; or water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure caused by or resulting from human or animal forces or any act of nature. Direct loss by fire, explosion or theft resulting from water damage is also covered. Additionally, the po

In [23]:
response = final_engine.query(
    """
    What are the differences in how the policy addresses water damage for dwelling damage and personal property damage?
    Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer.
    """
)
print(response.response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["\\n        Follow the example, but instead of giving a question, always prefix the question \\n        with: \'By first identifying and quoting the most relevant sources, \'. \\n        Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\

In [40]:
Markdown(f"{response.response}")

The policy addresses water damage for dwelling damage and personal property damage differently. For dwelling damage, the policy covers direct loss by fire, explosion or theft resulting from water damage under Section I – Property Coverages A. Coverage A – Dwelling, which states that "We cover: a.The dwelling on the "residence premises" shown in the Declarations, including structures attached to the dwelling; and b.Materials and supplies located on or next to the "residence premises" used to construct, alter, or repair the dwelling or other structures on the "residence premises". Additionally, Section I – Exclusions 3. Water Damage states that "Water Damage means: a.Flood, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind; b.Water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment; or c.Water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure; caused by or resulting from human or animal forces or any act of nature." Furthermore, Section I – Exclusions 5. Mold, Fungus Or Wet Rot states that "We do insure for loss caused by mold, fungus or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within: (a)A plumbing, heating, air conditioning or automatic fire protective sprinkler system, or a household appliance, on the "residence premises"; or (b)A storm drain, or water, steam or sewer pipes, off the "residence premises". For purposes of this provision, a plumbing system or household appliance does not include a sump, sump pump or related equipment or a roof drain, gutter, downspout or similar fixtures or equipment."

For personal property damage, the policy states that it will be settled at actual cash value at the time of loss, but not more than the amount required to repair or replace. Losses caused by weather conditions, acts or decisions of any person, group, organization or governmental body, and faulty, inadequate or defective planning, zoning, development, surveying, siting, design, specifications, workmanship, repair, construction, renovation, remodeling, grading, compaction, materials used in repair, construction, renovation or remodeling, or maintenance of part or all of any property are excluded from coverage.

Sources: 
Section I – Property Coverages A. Coverage A – Dwelling
Section I – Exclusions 3. Water Damage
Section I – Exclusions 5. Mold, Fungus Or Wet Rot

In [39]:
print(response.source_nodes[1])


node=TextNode(id_='4dd002cd-004d-472e-bb5e-8695c8bcb083', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='2e97ba3e2ce0b2dd702b8454b53d0fa172a5da72e5effd916ba2775c125c07bb', text='Sub question: By first identifying and quoting the most relevant sources, what does the policy say about water damage for personal property damage?\nResponse: \n\nThe policy states that "Water Damage means: a. Flood, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind; b. Water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment; or c. Water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure; caused by or resulting from human or animal 

In [214]:
from llama_index import Prompt

my_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question below. Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer. \n"
    "\n{query_str}\n"
)


In [215]:
CUSTOM_REFINE_PROMPT_TMPL = Prompt(
    "The original question is as follows: {query_str}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better answer the question. "
    "If the context isn't useful, return the original answer. "
    "Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer. "
    
)

In [216]:
from IPython.display import Markdown
from llama_index import (ServiceContext, 
                         StorageContext,
                         LLMPredictor,
                         load_index_from_storage)
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL
from llama_index.query_engine import SubQuestionQueryEngine, CitationQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata
from langchain.chat_models import ChatOpenAI
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.response_synthesizers import get_response_synthesizer


def get_llm_predictor(temperature=0):
    return LLMPredictor(ChatOpenAI(temperature=temperature, model="gpt-3.5-turbo"))


llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

service_context = ServiceContext.from_defaults(
    llm_predictor=get_llm_predictor(),
    callback_manager=callback_manager)


index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./_policy_index_storage"),
    )

query_engine = index.as_query_engine(
    refine_template=CUSTOM_REFINE_PROMPT_TMPL,
    text_qa_template=my_qa_template)

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="homeowners_policy", description="Homeowner's insurance policy document"
        ),
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=False,
)

DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\docstore.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\docstore.json.
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/docstore.json
open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/docstore.json
DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\index_store.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\index_store.json.
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/index_store.json
open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/index_store.json
DEBUG:llama_index.vector_stores.simple:Loading llama_index.vector_stores.simple 

In [217]:
response = await query_engine.aquery(
    "What are the considerations for determining if the policy does or does not cover damaged caused by collapse?"
)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\",\\n        \\"tool_name\\": \\"uber_10k\\"\\n    },\\n    {\\n        \\"sub_question\\": \\"What is the EBITDA of Uber\\",\\n        \\"tool_na

In [218]:
Markdown(f"{response.response}")

The considerations for determining if the policy covers damages caused by collapse are as follows:

1. The damage must be a direct result of the collapse of a building or any part of a building.
2. The policy covers loss to specific items such as awnings, fences, patios, decks, pavements, swimming pools, underground pipes, flues, drains, cesspools, septic tanks, foundations, retaining walls, bulkheads, piers, wharves, or docks if they are directly affected by the collapse.
3. The policy also covers breakage of glass or safety glazing material that is part of a covered building, storm door, or storm window, either as a result of the collapse or directly caused by earth movement.
4. The policy covers direct physical loss to covered property caused by a Peril Insured Against under Coverage A, excluding earthquake or land shock waves or tremors before, during, or after a volcanic eruption.

However, it is important to note that this coverage does not increase the limit of liability that applies to the damaged covered property.

In [219]:
Markdown(f"{response.source_nodes[0].node.get_text()}")

Sub question: What does the policy cover for damages caused by collapse
Response: 
The policy covers the following damages caused by collapse: 
(1) Loss to an awning, fence, patio, deck, pavement, swimming pool, underground pipe, flue, drain, cesspool, septic tank, foundation, retaining wall, bulkhead, pier, wharf or dock if the loss is a direct result of the collapse of a building or any part of a building (Section 8. Collapse, b.); 
(2) Breakage of glass or safety glazing material which is part of a covered building, storm door or storm window (Section 9. Glass Or Safety Glazing Material, a.(1)); 
(3) Breakage of glass or safety glazing material which is part of a covered building, storm door or storm window when caused directly by earth movement (Section 9. Glass Or Safety Glazing Material, a.(2)); 
(4) Direct physical loss to covered property caused by a Peril Insured Against under Coverage A, other than Earthquake or Land shock waves or tremors before, during or after a volcanic eruption (Section 9. Glass Or Safety Glazing Material, a.(3)). 

Sources: 
Section 8. Collapse, b. 
Section 9. Glass Or Safety Glazing Material, a.(1) 
Section 9. Glass Or Safety Glazing Material, a.(2) 
Section 9. Glass Or Safety Glazing Material, a.(3)

In [220]:
Markdown(f"{response.source_nodes[1].node.get_text()}")

Sub question: What does the policy not cover for damages caused by collapse
Response: 
The policy does not cover loss to an awning, fence, patio, deck, pavement, swimming pool, underground pipe, flue, drain, cesspool, septic tank, foundation, retaining wall, bulkhead, pier, wharf or dock caused by collapse, unless the loss is a direct result of the collapse of a building or any part of a building. This coverage does not increase the limit of liability that applies to the damaged covered property. 

Source: Sample_HO3_Policy, pages 7-7, c.

In [205]:
pp.pprint_response(response, show_source=True)

Final Response: The considerations for determining if the policy
covers damage caused by collapse include: 1. The collapse must be an
abrupt falling down or caving in of a building or any part of a
building, rendering it uninhabitable for its intended purpose. 2. The
collapse must be caused by one or more of the specified perils insured
against, such as perils under Coverage A, weight of people or personal
property, weight of snow, ice or sleet, etc. 3. The collapse must not
be caused by any excluded perils, such as earthquake, land shock waves
or tremors before, during or after a volcanic eruption, assessments
charged by governmental bodies, etc. 4. The policy period under
Section I - Conditions should also be considered. These factors need
to be assessed to determine if the policy provides coverage for damage
caused by collapse.
______________________________________________________________________
Source Node 1/2
Node ID: 53f99cfe-1c58-4b91-8e93-2dfad4462b7b
Similarity: None
Text: S

In [221]:
print(response.source_nodes[0].node.get_text())

Sub question: What does the policy cover for damages caused by collapse
Response: 
The policy covers the following damages caused by collapse: 
(1) Loss to an awning, fence, patio, deck, pavement, swimming pool, underground pipe, flue, drain, cesspool, septic tank, foundation, retaining wall, bulkhead, pier, wharf or dock if the loss is a direct result of the collapse of a building or any part of a building (Section 8. Collapse, b.); 
(2) Breakage of glass or safety glazing material which is part of a covered building, storm door or storm window (Section 9. Glass Or Safety Glazing Material, a.(1)); 
(3) Breakage of glass or safety glazing material which is part of a covered building, storm door or storm window when caused directly by earth movement (Section 9. Glass Or Safety Glazing Material, a.(2)); 
(4) Direct physical loss to covered property caused by a Peril Insured Against under Coverage A, other than Earthquake or Land shock waves or tremors before, during or after a volcanic e

In [222]:
print(response.source_nodes[1].node.get_text())

Sub question: What does the policy not cover for damages caused by collapse
Response: 
The policy does not cover loss to an awning, fence, patio, deck, pavement, swimming pool, underground pipe, flue, drain, cesspool, septic tank, foundation, retaining wall, bulkhead, pier, wharf or dock caused by collapse, unless the loss is a direct result of the collapse of a building or any part of a building. This coverage does not increase the limit of liability that applies to the damaged covered property. 

Source: Sample_HO3_Policy, pages 7-7, c.


In [223]:
from llama_index.callbacks.schema import CBEventType, EventPayload

for start_event, end_event in llama_debug.get_event_pairs(CBEventType.SUB_QUESTIONS):
    for i, qa_pair in enumerate(end_event.payload[EventPayload.SUB_QUESTIONS]):
        print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
        print("Answer: " + qa_pair.answer.strip())
        print("====================================")

Sub Question 0: What does the policy cover for damages caused by collapse
Answer: The policy covers the following damages caused by collapse: 
(1) Loss to an awning, fence, patio, deck, pavement, swimming pool, underground pipe, flue, drain, cesspool, septic tank, foundation, retaining wall, bulkhead, pier, wharf or dock if the loss is a direct result of the collapse of a building or any part of a building (Section 8. Collapse, b.); 
(2) Breakage of glass or safety glazing material which is part of a covered building, storm door or storm window (Section 9. Glass Or Safety Glazing Material, a.(1)); 
(3) Breakage of glass or safety glazing material which is part of a covered building, storm door or storm window when caused directly by earth movement (Section 9. Glass Or Safety Glazing Material, a.(2)); 
(4) Direct physical loss to covered property caused by a Peril Insured Against under Coverage A, other than Earthquake or Land shock waves or tremors before, during or after a volcanic er

In [224]:
query = "What are the duties owed of the policy holder and for the insurance company?"

response = response = await query_engine.aquery(query)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\",\\n        \\"tool_name\\": \\"uber_10k\\"\\n    },\\n    {\\n        \\"sub_question\\": \\"What is the EBITDA of Uber\\",\\n        \\"tool_na

In [225]:
Markdown(f"{response.response}")

The duties owed by the policy holder are:
1. Give written notice to the insurance company or their agent as soon as possible, providing information about the policy, the occurrence, and the names and addresses of any claimants and witnesses.
2. Cooperate with the insurance company in the investigation, settlement, or defense of any claim or lawsuit.
3. Promptly forward any notices, demands, summonses, or other legal processes related to the occurrence to the insurance company.
4. Assist the insurance company upon request in making settlements, enforcing rights of contribution or indemnity, conducting suits, attending hearings and trials, and securing evidence and witnesses.
5. Give prompt notice to the insurance company or their agent.
6. Notify the police in case of loss by theft.
7. Notify the credit card or electronic fund transfer card or access device company in case of loss as specified in the policy.
8. Take steps to protect the property from further damage.

The duties owed by the insurance company are:
1. Give written notice to the policy holder or their agent as soon as possible, providing information about the policy, the occurrence, and the names and addresses of any claimants and witnesses.
2. Cooperate with the policy holder in the investigation, settlement, or defense of any claim or lawsuit.
3. Promptly receive and review any notices, demands, summonses, or other legal processes related to the occurrence.
4. Assist the policy holder upon request in making settlements, enforcing rights of contribution or indemnity, conducting suits, attending hearings and trials, and securing evidence and witnesses.
5. With respect to damage to property, receive prompt notice from the policy holder, cooperate with the policy holder in notifying the police in case of theft, and cooperate with the policy holder in notifying the credit card or electronic fund transfer card or access device company in case of loss as specified in the policy.
6. Take steps to protect the property from further damage.

In [226]:
print(response.source_nodes[0].node.get_text())

Sub question: What are the duties owed by the policy holder?
Response: 
1. Give written notice to us or our agent as soon as is practical, which sets forth: a. The identity of the policy and the "named insured" shown in the Declarations; b. Reasonably available information on the time, place and circumstances of the "occurrence"; and c. Names and addresses of any claimants and witnesses;
2. Cooperate with us in the investigation, settlement or defense of any claim or suit;
3. Promptly forward to us every notice, demand, summons or other process relating to the "occurrence";
4. At our request, help us: a. To make settlement; b. To enforce any right of contribution or indemnity against any person or organization who may be liable to an "insured"; c. With the conduct of suits and attend hearings and trials; and d. To secure and give evidence and obtain the attendance of witnesses;
5. Give prompt notice to us or our agent;
6. Notify the police in case of loss by theft;
7. Notify the credit

In [228]:
Markdown(f"{response.source_nodes[0].node.text}")

Sub question: What are the duties owed by the policy holder?
Response: 
1. Give written notice to us or our agent as soon as is practical, which sets forth: a. The identity of the policy and the "named insured" shown in the Declarations; b. Reasonably available information on the time, place and circumstances of the "occurrence"; and c. Names and addresses of any claimants and witnesses;
2. Cooperate with us in the investigation, settlement or defense of any claim or suit;
3. Promptly forward to us every notice, demand, summons or other process relating to the "occurrence";
4. At our request, help us: a. To make settlement; b. To enforce any right of contribution or indemnity against any person or organization who may be liable to an "insured"; c. With the conduct of suits and attend hearings and trials; and d. To secure and give evidence and obtain the attendance of witnesses;
5. Give prompt notice to us or our agent;
6. Notify the police in case of loss by theft;
7. Notify the credit card or electronic fund transfer card or access device company in case of loss as provided for in E.6. Credit Card, Electronic Fund Transfer Card Or Access Device, Forgery And Counterfeit Money under Section I – Property Coverages;
8. Protect the property from further damage.

In [229]:
Markdown(f"{response.source_nodes[1].node.text}")

Sub question: What are the duties owed by the insurance company?
Response: 
The duties owed by the insurance company are: 
1. Give written notice to us or our agent as soon as is practical, which sets forth: 
a. The identity of the policy and the "named insured" shown in the Declarations; 
b. Reasonably available information on the time, place and circumstances of the "occurrence"; and 
c. Names and addresses of any claimants and witnesses; 
2. Cooperate with us in the investigation, settlement or defense of any claim or suit; 
3. Promptly forward to us every notice, demand, summons or other process relating to the "occurrence"; 
4. At our request, help us: 
a. To make settlement; 
b. To enforce any right of contribution or indemnity against any person or organization who may be liable to an "insured"; 
c. With the conduct of suits and attend hearings and trials; and 
d. To secure and give evidence and obtain the attendance of witnesses; 
5. With respect to C. Damage To Property, give prompt notice to us or our agent, notify the police in case of loss by theft, and notify the credit card or electronic fund transfer card or access device company in case of loss as provided for in E.6. Credit Card, Electronic Fund Transfer Card Or Access Device, Forgery And Counterfeit Money under Section I – Property Coverages; 
6. Protect the property from further damage.